In [5]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from IPython.display import Image, display # For displaying the graph
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [8]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [3]:
class blogState(TypedDict):
    topic : str
    outline : str
    blog_content : str

In [9]:
def outline_gen(state: blogState) -> blogState:
    topic = state["topic"]
    template = "Generate an outline for a blog on the topic: {topic}"
    prompt = PromptTemplate.from_template(template)
    state["outline"] = llm.invoke(prompt.format(topic=topic)).content
    return state

In [10]:
def blog_gen(state: blogState) -> blogState:
    topic = state["topic"]
    outline = state["outline"]
    template = "Generate a blog for the given topic : {topic} and \n\n Outline : {outline}"
    prompt = PromptTemplate.from_template(template)
    state["blog_content"] = llm.invoke(prompt.format(topic=topic,outline=outline)).content
    return state

In [11]:
graph = StateGraph(blogState)

graph.add_node("create_outline", outline_gen)
graph.add_node("create_blog", blog_gen)

graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", END)

workflow = graph.compile()

In [ ]:
initial_state = {"topic": "Will AI be taking your Jobs in the coming future?"}
final_state = workflow.invoke(initial_state)

In [13]:
print(final_state)

{'topic': 'Will AI be taking your Jobs in the coming future?', 'outline': 'Here\'s a comprehensive outline for a blog post addressing the question of AI taking jobs, aiming for a balanced and informative perspective.\n\n---\n\n## Blog Outline: Will AI Be Taking Your Jobs in the Coming Future?\n\n**Working Title Options:**\n*   AI and Your Job: Hype vs. Reality – What You Need to Know\n*   The AI Job Apocalypse? Understanding How AI Will Reshape the Workforce\n*   Future of Work: Is AI Coming for Your Job, or Just Transforming It?\n*   Navigating the AI Era: Your Job, Your Future, and the Rise of Intelligent Machines\n\n---\n\n### I. Introduction (The Hook & The Thesis)\n\n*   **A. The Fear Factor (Hook):**\n    *   Start with the common narrative: headlines screaming about robots taking over, job losses, and economic disruption.\n    *   Acknowledge the widespread anxiety and speculation.\n    *   *Example:* "Is the robot apocalypse upon us? Every day, it seems there\'s another headlin

In [14]:
print(final_state["outline"])

Here's a comprehensive outline for a blog post addressing the question of AI taking jobs, aiming for a balanced and informative perspective.

---

## Blog Outline: Will AI Be Taking Your Jobs in the Coming Future?

**Working Title Options:**
*   AI and Your Job: Hype vs. Reality – What You Need to Know
*   The AI Job Apocalypse? Understanding How AI Will Reshape the Workforce
*   Future of Work: Is AI Coming for Your Job, or Just Transforming It?
*   Navigating the AI Era: Your Job, Your Future, and the Rise of Intelligent Machines

---

### I. Introduction (The Hook & The Thesis)

*   **A. The Fear Factor (Hook):**
    *   Start with the common narrative: headlines screaming about robots taking over, job losses, and economic disruption.
    *   Acknowledge the widespread anxiety and speculation.
    *   *Example:* "Is the robot apocalypse upon us? Every day, it seems there's another headline warning that AI is coming for our jobs, leaving many wondering: should I be worried?"

*   **B